In [1]:
# import slicer
# slicer.util.pip_install("natsort")
# slicer.util.pip_install("nibabel")
# slicer.util.pip_install("tqdm")

In [2]:
import os
from glob import glob
from natsort import natsorted
import numpy as np
import subprocess
import nibabel as nib
import json
from tqdm import tqdm
import pandas as pd

In [3]:
# define the target image for the registration and the corresponding face mask
target = os.path.abspath("./data/icbm152_ext55_model_sym_2020_nifti/icbm152_ext55_model_sym_2020/mni_icbm152_t1_tal_nlin_sym_55_ext.nii")
target_face_mask = os.path.abspath("./data/icbm152_ext55_model_sym_2020_nifti/icbm152_ext55_model_sym_2020/t1_mask.nii.gz")

# define the list iof images that need to be defaced
floating_imgs = natsorted(glob("./data/example_input_images/*/*.nii.gz"))

# define a list of existing transforms (for example created with 3D Slicer) that are used instead of the automatic registration
# value should be None if no transform exists and automatic registration is to be used
existing_transform_paths = [os.path.join(os.path.dirname(f), "Transform_to_template.txt") for f in floating_imgs]
existing_transform_paths = [f if os.path.isfile(f) else "None" for f in existing_transform_paths ]

# list of output paths for each defaced image
results_folder_paths = [os.path.join("./data/example_output/defaced_images", *f.split(os.sep)[-2:-1]) for f in floating_imgs]

# check that all lists have the same length
assert len(floating_imgs) == len(results_folder_paths) == len(existing_transform_paths), f"The lists have different lengths, {len(floating_imgs)=}, {len(results_folder_paths)=}, {len(existing_transform_paths)=}"

print(f"Found {len(floating_imgs)} images that need to be defaced")

# print example paths
print("Input images: ", floating_imgs[0:3])
print("Existing transforms: ", existing_transform_paths[0:3])
print("Output paths: ", results_folder_paths[0:3])

# path to BRAINSFit executable
BRAINSFit_bin_path = os.path.join("./BRAINSTools/BRAINSFit")

# path to BRAINSFit executable
BRAINSresample_bin_path = "./BRAINSTools/BRAINSResample"

Found 2 images that need to be defaced
Input images:  ['./data/example_input_images/IXI002-Guys-0828-T1/IXI002-Guys-0828-T1.nii.gz', './data/example_input_images/IXI002-Guys-0828-T2/IXI002-Guys-0828-T2.nii.gz']
Existing transforms:  ['None', 'None']
Output paths:  ['./data/example_output/defaced_images/IXI002-Guys-0828-T1', './data/example_output/defaced_images/IXI002-Guys-0828-T2']


In [4]:
for i, (floating, results_folder_path, ex_tfm) in enumerate(tqdm(list(zip(floating_imgs, results_folder_paths, existing_transform_paths)))):
    print(i)

    if os.path.isfile(os.path.join(results_folder_path, os.path.basename(floating).replace(".nii.gz", "_masked.nii.gz"))):
        print("file exists. skipping ..... ")
        continue
    else:
        os.makedirs(results_folder_path, exist_ok=True)

        out_affine = os.path.join(results_folder_path, os.path.basename(floating.replace("nii.gz", "txt")))
        out_resampled_img_path = os.path.join(results_folder_path, os.path.basename(floating).replace(".nii.gz", "_resampled.nii.gz"))
        
        target_nii = nib.load(target)
        floating_nii = nib.load(floating)
        
        if os.path.isfile(ex_tfm):
            print(f"Using existing initialTransform {ex_tfm} instead of running registration...")
            out_affine = ex_tfm
        else:
            print(f"Run registration with BRAINSFit...")
            exit_status = os.system(f' "{BRAINSFit_bin_path}" ' +  # BRAINSFit executable path
                                    f'--fixedVolume "{target}" ' +  # Path to the fixed (target) volume
                                    f'--movingVolume "{floating}" ' +  # Path to the moving (floating) volume
                                    f'--outputVolume "{out_resampled_img_path}" ' +  # Path to the output resampled volume
                                    f'--outputTransform "{out_affine}" ' +  # Path to the output transformation matrix
                                    f'--samplingPercentage 0.1  ' +  # 10% of the data used for registration (lower values = faster, less accurate)
                                    f'--splineGridSize 14,10,12  ' +  # Grid size for spline transformation
                                    f'--initializeTransformMode useMomentsAlign  ' +  # Method to initialize the transformation
                                    f'--useRigid  ' +  # Apply rigid transformation (translation, rotation)
                                    f'--useAffine  ' +  # Apply affine transformation (scaling, shearing)
                                    f'--maskProcessingMode NOMASK  ' +  # Don't apply any masks during registration
                                    f'--medianFilterSize 0,0,0  ' +  # Disable median filtering (default 0)
                                    f'--removeIntensityOutliers 0  ' +  # Disable outlier removal
                                    f'--outputVolumePixelType float  ' +  # Set output pixel type to float
                                    f'--backgroundFillValue 0  ' +  # Background fill value set to 0 (empty space in images)
                                    f'--interpolationMode Linear  ' +  # Use linear interpolation for image resampling
                                    f'--numberOfIterations 1500  ' +  # Number of iterations for optimization
                                    f'--maximumStepLength 0.05  ' +  # Maximum step size for optimization
                                    f'--minimumStepLength 0.001  ' +  # Minimum step size for optimization
                                    f'--relaxationFactor 0.5  ' +  # Relaxation factor for optimization
                                    f'--translationScale 1000  ' +  # Scale for translation optimization
                                    f'--reproportionScale 1  ' +  # Scale for reproportioning optimization
                                    f'--skewScale 1  ' +  # Scale for skewness optimization
                                    f'--maxBSplineDisplacement 0  ' +  # Maximum displacement for B-spline transformation
                                    f'--fixedVolumeTimeIndex 0  ' +  # Time index for the fixed volume (0 means no time series)
                                    f'--movingVolumeTimeIndex 0  ' +  # Time index for the moving volume (0 means no time series)
                                    f'--numberOfHistogramBins 50  ' +  # Number of histogram bins for image matching
                                    f'--numberOfMatchPoints 10  ' +  # Number of points used in matching process
                                    f'--costMetric MMI  ' +  # Use Mutual Information (MMI) as the cost metric for registration
                                    f'--maskInferiorCutOffFromCenter 1000  ' +  # Masking option (cut off below 1000 units from center)
                                    f'--ROIAutoDilateSize 0  ' +  # Dilate ROI automatically (disabled)
                                    f'--ROIAutoClosingSize 9  ' +  # Apply morphological closing (dilation + erosion)
                                    f'--numberOfSamples 0  ' +  # Number of samples used in cost function (0 for all samples)
                                    f'--failureExitCode -1  ' +  # Exit code on failure
                                    f'--numberOfThreads -1  ' +  # Use all available threads
                                    f'--debugLevel 0  ' +  # Debug level (0 = no debug)
                                    f'--costFunctionConvergenceFactor 2e+13  ' +  # Convergence factor for cost function
                                    f'--projectedGradientTolerance 1e-05  ' +  # Tolerance for projected gradient
                                    f'--maximumNumberOfEvaluations 900  ' +  # Maximum number of evaluations for optimization
                                    f'--maximumNumberOfCorrections 25  ' +  # Maximum number of corrections for optimization
                                    f'--metricSamplingStrategy Random '  # Randomly sample points for metric calculation
                                    f'>> /dev/null'  # Suppress output to the terminal
                                       )
        
            # Check if the exit status is zero (success)
            if exit_status == 0:
                print("BRAINSFit registration completed successfully!")
            else:
                print(f"BRAINSFit failed with exit code {exit_status}. Please check for errors.")
        
        # transform face mask to space of the floating image
        os.system(f'./apply_affine "{target_face_mask}" "{floating}" "{out_affine}" "{results_folder_path}" -noshow')
        
        # define path to the registered and resampled face mask
        face_mask_resampled_path = os.path.join(results_folder_path, os.path.basename(target_face_mask).replace(".nii.gz", "_resampled.nii.gz"))
        
        os.system(      f' "{BRAINSresample_bin_path}" '  + 
                        f'--inputVolume "{target_face_mask}" ' + 
                        f' --referenceVolume "{floating}" ' +
                        f'--outputVolume "{face_mask_resampled_path}" ' + 
                        f'--warpTransform "{out_affine}" ' + 
                        f'--inverseTransform ' +
                        f'--interpolationMode NearestNeighbor '
                        f'>> /dev/null'
                )

        # apply mask to image
        floating_masked_path = os.path.join(results_folder_path, os.path.basename(floating).replace(".nii.gz", "_masked.nii.gz"))

        floating_nii = nib.load(floating)
        floating_data = floating_nii.get_fdata()

        mask_nii = nib.load(face_mask_resampled_path)
        mask_data = mask_nii.get_fdata()

        floating_masked_data = floating_data * mask_data
        floating_masked_nii = nib.Nifti1Image(floating_masked_data, affine = floating_nii.affine)

        nib.save(floating_masked_nii, floating_masked_path)

  0%|          | 0/2 [00:00<?, ?it/s]

0


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


Run registration with BRAINSFit...
BRAINSFit registration completed successfully!


 50%|#####     | 1/2 [01:14<01:14, 74.96s/it]

1


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


Run registration with BRAINSFit...
BRAINSFit registration completed successfully!


100%|##########| 2/2 [04:17<00:00, 128.68s/it]
